# Kaggle: Red Wine Quality (Cortez et al. 2009)

이 노트북은 Kaggle의 Red Wine Quality 데이터셋(P. Cortez et al., 2009)을 활용하여 레드 와인의 품질을 예측하는 모델을 만드는 과정을 담고 있습니다.
데이터셋은 UCI Machine Learning Repository에서 직접 다운로드하여 사용합니다.

## 데이터셋 설명 (Dataset Description)

- **데이터셋**: Red Wine Quality Dataset
- **출처**: [Kaggle Link](https://www.kaggle.com/datasets/uciml/red-wine-quality-cortez-et-al-2009) / [UCI Link](https://archive.ics.uci.edu/ml/datasets/wine+quality)
- **데이터 개수**: 1,599개
- **특성(Features)**: 11개 (화학적 특성)
- **타겟(Target)**: quality (0에서 10 사이의 점수)

### 변수 설명 (Variables)
1. **fixed acidity** (고정 산도): 와인의 산도와 관련된 고정된 산의 양
2. **volatile acidity** (휘발성 산도): 와인에서 식초 맛을 내는 휘발성 산의 양 (너무 높으면 좋지 않음)
3. **citric acid** (구연산): 와인의 신선함을 더해주는 산
4. **residual sugar** (잔류 당분): 발효 후 남은 당분 (와인의 단맛 결정)
5. **chlorides** (염화물): 와인 내 소금의 양
6. **free sulfur dioxide** (유리 이산화황): 미생물 번식과 산화를 방지하는 SO2 형태
7. **total sulfur dioxide** (총 이산화황): 유리형과 결합형 SO2의 총합
8. **density** (밀도): 알코올 함량과 당분에 따라 달라지는 와인의 밀도
9. **pH**: 와인의 산성도 (0: 강산성 ~ 14: 강알칼리성, 보통 3-4 사이)
10. **sulphates** (황산염): 이산화황 농도에 영향을 주어 항균 및 항산화 작용을 함
11. **alcohol** (알코올 도수): 와인의 알코올 함량 (% vol)
12. **quality** (품질): 출력 변수 (0~10 사이의 점수, 3이 가장 낮고 8이 가장 높음)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import urllib.request

# 전처리 라이브러리
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.preprocessing import StandardScaler, LabelEncoder

# 머신러닝 모델
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, ExtraTreesClassifier, VotingClassifier, AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
# 평가 지표
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, f1_score

import warnings
warnings.filterwarnings('ignore')

## 1. 데이터 로드 (Data Load)

Kaggle에서 데이터를 직접 다운로드합니다.

In [ ]:
# kaggle에 로그인을 해서 데이터셋을 다운로드 받기 위해
# 로그인을 위한 키값을 설정
from dotenv import load_dotenv
# .env로 부터 파일을 읽으면 True가 출력
print(load_dotenv())

In [ ]:
#!/bin/bashkaggle
!kaggle datasets download uciml/red-wine-quality-cortez-et-al-2009

In [ ]:
import zipfile
import os
# 압축 파일 경로 (정방향 슬래시 사용)
zip_file_path = 'C:/Users/park/iCloudDrive/2025/EST_AI_MODEL_DEVEL/ESTSoft/DataScience/scikit-learn/red-wine-quality-cortez-et-al-2009.zip'
# 압축을 풀 대상 디렉토리
extract_to_path = 'C:/Users/park/iCloudDrive/2025/EST_AI_MODEL_DEVEL/ESTSoft/DataScience/scikit-learn/data'

# zip 파일 열기 및 압축 해제
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to_path)
    print(f"압축 해제 완료: {extract_to_path}")
# 해제된 파일 목록 확인
print("현재 폴더 파일 목록:", os.listdir(extract_to_path))


In [ ]:
import pandas as pd

filename = os.path.join(extract_to_path, 'winequality-red.csv')
# 세미콜론(;)으로 구분된 CSV 파일
df = pd.read_csv(filename)
df.head()

## 2. 데이터 분석 (EDA)

In [ ]:
print("### Data Info ###")
print(df.info())
# 다 수치형이면 전처리가 쉽다.

In [ ]:
print("\n### Basic Statistics ###")
display(df.describe().round(4))

In [ ]:
df['quality'].unique()

In [ ]:
# 타겟 변수(Quality) 분포 확인
plt.figure(figsize=(8, 5))
sns.countplot(x='quality', data=df, palette='viridis')
plt.title("Red Wine Quality Distribution")
plt.show()

# 상관관계 히트맵
plt.figure(figsize=(10, 8))
corr = df.corr()
sns.heatmap(corr, annot=True, fmt='.2f', cmap='coolwarm', square=True)
plt.title("Correlation Matrix")
plt.show()

# quality 3, 4, 8은 데이터 수가 너무 작아 예측이 힘들다.

## 3. 데이터 전처리 및 특성 엔지니어링

- **특성 엔지니어링**: 산도 관련 특성들을 합쳐서 `total acidity`를 만들어 봅니다 (선택 사항).
- **이진 분류 변환 (선택)**: 원래 점수(0~10)를 그대로 쓸 수도 있지만, 보통 좋은 와인(예: 7 이상)과 나쁜 와인으로 나누거나 그대로 멀티 클래스로 분류합니다. 여기서는 **멀티 클래스 분류(각 점수별 예측)**를 수행하되, 데이터가 적은 클래스(3, 8 등)를 고려해야 합니다. 
  > 여기서는 **quality 그대로를 클래스로 사용하는 멀티클래스 분류**로 진행합니다.

- **스케일링**: 대부분의 모델(SVM, KNN, Logistic 등)은 특성 스케일에 민감하므로 `StandardScaler`를 적용합니다.

In [ ]:
# 특성 엔지니어링 (예: 총 산도)
# df['total_acidity'] = df['fixed acidity'] + df['volatile acidity'] + df['citric acid']
# 이 예제에서는 원본 특성을 유지하며 진행합니다.

X = df.drop('quality', axis=1)
y = df['quality']

# 타겟 클래스 개수가 적은 경우 (예: quality 3, 8) Stratified Split 사용 권장
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# 스케일링
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f"Train shape: {X_train_scaled.shape}, Test shape: {X_test_scaled.shape}")

In [ ]:
# y_train에서 숫자별 데이터의 갯수 확인
print(y_train.value_counts())
print(y_test.value_counts())

In [ ]:
!pip install imbalanced-learn

In [ ]:
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train_scaled, y_train)

In [ ]:
print(pd.Series(y_train[:]).value_counts())
print(pd.Series(y_train_res[:]).value_counts())

## 4. 모델링 (8가지 모델)

In [ ]:
models = {
    'LogisticRegression': LogisticRegression(max_iter=2000, random_state=42),
    'DecisionTree': DecisionTreeClassifier(random_state=42),
    'RandomForest': RandomForestClassifier(random_state=42, class_weight='balanced_subsample'),
    'SVM': SVC(probability=True, random_state=42),
    'KNN': KNeighborsClassifier(),
    'GradientBoosting': GradientBoostingClassifier(random_state=42),
    'ExtraTrees': ExtraTreesClassifier(random_state=42),
    'AdaBoost': AdaBoostClassifier(random_state=42)
}

results = {}
print("### Initial Model Performance (Accuracy) ###")
for name, model in models.items():
    #model.fit(X_train_scaled, y_train)
    model.fit(X_train_res, y_train_res)
    y_pred = model.predict(X_test_scaled)
    acc = accuracy_score(y_test, y_pred)
    results[name] = acc
    print(f"{name}: {acc:.4f}")

# 상위 4개 모델 선정
sorted_results = sorted(results.items(), key=lambda x: x[1], reverse=True)
top_4_names = [item[0] for item in sorted_results[:4]]
print(f"\nTop 4 Models: {top_4_names}")

## 5. 하이퍼 파라메터 튜닝 및 앙상블

상위 4개 모델에 대해 `GridSearchCV` 또는 간단한 튜닝을 수행한 후, `VotingClassifier`로 앙상블합니다.

In [ ]:
# Top 4 모델 객체 가져오기
top_models = {}
for name in top_4_names:
    top_models[name] = models[name]

# 튜닝을 위한 파라미터 그리드 예시 (시간 관계상 일부만 간단히 설정)
param_grids = {
    'RandomForest': {'n_estimators': [50, 100, 200, 300], 'max_depth': [10, 20, 25, None], 'class_weight':['balanced','balanced_subsample',None]},
    'ExtraTrees': {'n_estimators': [50, 70, 100, 200], 'max_depth': [10, 20, 25, 30, None]},
    'SVM': {'C': [1, 10], 'gamma': ['scale', 0.1]},
    'GradientBoosting': {'n_estimators': [50, 75, 100, 200], 'learning_rate': [0.03, 0.05, 0.1]},
    'KNN': {'n_neighbors': [3, 5, 7]},
    'DecisionTree': {'max_depth': [5, 10, None]},
    'LogisticRegression': {'C': [1, 10]},
    'AdaBoost': {'n_estimators': [50, 100]}
}

tuned_estimators = []

print("\n### Hyperparameter Tuning for Top 4 ###")
for name in top_4_names:
    if name in param_grids:
        grid = GridSearchCV(top_models[name], param_grids[name], cv=3, n_jobs=-1, scoring='accuracy')
        grid.fit(X_train_res, y_train_res)
        best_model = grid.best_estimator_
        print(f"{name} Best Params: {grid.best_params_}, Score: {grid.best_score_:.4f}")
        tuned_estimators.append((name, best_model))
    else:
        # 튜닝할 파라미터가 없거나 설정 안된 경우 기본 모델 사용
        tuned_estimators.append((name, top_models[name]))

# 앙상블 모델 생성 (Hard Voting)
ensemble_model = VotingClassifier(estimators=tuned_estimators, voting='hard')
ensemble_model.fit(X_train_res, y_train_res)

ensemble_pred = ensemble_model.predict(X_test_scaled)
ensemble_acc = accuracy_score(y_test, ensemble_pred)

print(f"\nEnsemble Model Test Accuracy: {ensemble_acc:.4f}")

In [ ]:
# 튜닝된 모델 중 가장 성능이 좋은 모델 찾기
best_tuned_name = None
best_tuned_score = 0
best_tuned_model = None

for name, model in tuned_estimators:
    y_pred = model.predict(X_test_scaled)
    acc = accuracy_score(y_test, y_pred)
    print(f"{name} Test Accuracy: {acc:.4f}")
    if acc > best_tuned_score:
        best_tuned_score = acc
        best_tuned_name = name
        best_tuned_model = model

print(f"\n최고 성능 모델: {best_tuned_name} (Accuracy: {best_tuned_score:.4f})")

# 최고 성능 모델의 Confusion Matrix 시각화
best_pred = best_tuned_model.predict(X_test_scaled)
labels = sorted(y_test.unique())

print(f"\n### Confusion Matrix - {best_tuned_name} ###")
cm_best = confusion_matrix(y_test, best_pred)

plt.figure(figsize=(8, 6))
sns.heatmap(cm_best, 
            annot=True, 
            fmt='d', 
            cmap='Greens',
            xticklabels=labels,
            yticklabels=labels)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title(f'Confusion Matrix - Best Model ({best_tuned_name})')
plt.show()

print(f"\n### Classification Report - {best_tuned_name} ###")
print(classification_report(y_test, best_pred))

## 6. 모델 평가 및 결론

In [ ]:
# 실제 클래스 라벨 추출 (3, 4, 5, 6, 7, 8)
labels = sorted(y_test.unique())

print("### Confusion Matrix ###")
cm = confusion_matrix(y_test, ensemble_pred)

plt.figure(figsize=(8, 6))
sns.heatmap(cm, 
            annot=True, 
            fmt='d', 
            cmap='Blues',
            xticklabels=labels,  # x축에 3~8 표시
            yticklabels=labels)  # y축에 3~8 표시
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix - Ensemble Model')
plt.show()

In [ ]:
print(y_train.value_counts())
print(y_test.value_counts())